<a href="https://colab.research.google.com/github/Lorxus/Miscellaneous/blob/main/EpochTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import sympy as sp
import random as rand

MIN = 10**19
MAX = MIN + 10001
# ARRAY = range(MIN, MAX)
# flags = [False] * 10001
count = 0
tally = 0
exceptionalokprimecount = 0
exceptionalokprimecliplist = []

lookuptableofokprimes = [31, 43, 109, 127, 157, 223, 229, 277, 283, 307, 397, 433, 439, 457, 499, 601, 643, 691, 727, 733, 739, 811, 919, 997, 1021, 1051, 1069, 1093, 1327, 1399, 1423, 1459, 1471, 1579, 1597, 1627, 1657, 1699, 1723, 1753, 1777, 1789, 1801, 1831, 1933, 1999, 2017, 2347, 2749, 2767, 2791, 2917, 3163, 3229, 3259, 3271, 3463, 3823, 4027, 4423, 4519, 4549, 4567, 4597, 4651, 4663, 4759, 4909, 5101, 5167, 5413, 5437, 5503, 5653]  # the first few primes which are == 1 mod 3 and of form u^2 + 27v^2 for u, v \in Z

for u in range(1, 3162):
    if u%3 == 0:
        continue
    for v in range(1, 610):
        temp = u*u + 27*v*v
        if sp.isprime(temp) and temp%3 == 1 and temp not in lookuptableofokprimes:
            # print('Found a missing small ok prime! It\'s:', temp)
            lookuptableofokprimes.append(temp)

print('Small ok primes dictionary loaded...')

primelist = []  # this will have all the 1 mod 3 primes in the range
# thirdlist = []
for number in range(MIN, MAX):
    # by Landau (a(n) ~ 6n ln(n); n ~ 4.3 * 10^16) so that we should get ok primes ~every 139 (= 6 * 16ln(4.3))
    # such that we predict seeing ~71 such primes for MIN <= N <= MAX; we get 52 after ~1h
    if sp.isprime(number) and number % 3 == 1:
        primelist.append(number)
        # thirdlist.append(int((number-1)/3))
print('The number of =1mod3 primes between 10^19 and 10^19 + 10^4 is', len(primelist), '.')

XsqPlusTwentysevenYsq = []  # this will have the numbers that can be put in that form, from the range
for x in range(3162277660, 0, -1):
    if x%3 == 0:  # if so, can't be prime so don't bother
        continue
    templowidx = int(((MIN - x*x)/27)**0.5)
    temphighidx = int(((MAX - x*x)/27)**0.5)
    for y in range(templowidx, temphighidx + 1):  # this tops out at 608,580,619
        # if sp.gcd(x, y) != 1:  # this makes the code really really slow (est 16-20h)
        #     continue
        temp = x*x + 27*y*y  # in the formula above, is the answer even inside the desired range?
        if temp < MIN:  # too small, keep moving
            continue
        elif temp > MAX:  # too big, move on to the next x
            break
        elif temp not in XsqPlusTwentysevenYsq:
            XsqPlusTwentysevenYsq.append(temp)
            tempclip = temp - MIN
            if not sp.isprime(temp) and rand.random() < 0.01625:
                print('A new exceptional OK number! u =', x, '; v =', y, '; n = u^2 + 27v^2 = 10^19 +', tempclip)
            if sp.isprime(temp):
                print('A new exceptional OK number! u =', x, '; v =', y, '; n = u^2 + 27v^2 = 10^19 +', tempclip)
                exceptionalokprimecount += 1
                print('Oh dang and it\'s even a prime! Nice one, 10^19 +', tempclip, '!')
                exceptionalokprimecliplist.append(tempclip)
                exceptionalokprimecliplist = sorted(exceptionalokprimecliplist)
                print('Exceptional OK prime count:', exceptionalokprimecount)
                lookuptableofokprimes.append(temp)

lookuptableofokprimes = sorted(lookuptableofokprimes)
XsqPlusTwentysevenYsq = sorted(XsqPlusTwentysevenYsq)
print(len(XsqPlusTwentysevenYsq))
print('There are', exceptionalokprimecount, 'exceptional OK primes between 10^19 and 10^19 + 10^4.')
print('These are given by 10^19 +:', exceptionalokprimecliplist)
# assert False

def isprimegood(p: int):
    # checks whether or not a prime is good
    # good primes consist of 3 and all == 2 mod 3
    return sp.isprime(p) and (p == 3 or (p%3 == 2))

def isprimeok(p: int):
    # checks whether or not p is an ok prime, that is, equal to u^2 + 27v^2 for some integral u, v
    # p should never not be a prime that's == 1 mod 3
    if not sp.isprime(p) or not p%3 == 1:
        return False
    elif p in lookuptableofokprimes:
        return True
    else:
        okness = False
        # tempbit = int((p-1)/3)
        # tempres = (2**tempbit) % p
        # if tempres == -1:
        #     okness = True
        return okness

def isprimefineiguess(p: int):
    return isprimegood(p) or isprimeok(p)

def allgoodfactors(n: int):
    # checks whether all the factors of n are good primes
    if n == 1:
        return True
    if sp.isprime(n):
        return isprimefineiguess(n)
    else:
        if n%7 == 0 or n%13 == 0 or n%19 == 0 or n%37 == 0:
            return False
        if n%2 == 0:
            #print('Removed a 2!')
            return allgoodfactors(int(n/2))
        if n%3 == 0:
            #print('Removed a 3!')
            return allgoodfactors(int(n/3))
        if n%5 == 0:
            #print('Removed a 5!')
            return allgoodfactors(int(n/5))
        else:
            tempbool = True
            factorarray = sp.primefactors(n)
            for prime in factorarray:
                # print('Testing prime:', prime)
                tempbool = tempbool and isprimefineiguess(prime)

# print('10000000000000000336:', sp.primefactors(10000000000000000336))
# assert False

for number in range(MIN, MAX):
    tally += 1
    if rand.random() < 0.015625:
        print(tally)
    if number in lookuptableofokprimes:
        count += 1
        print('Success on', number, '!')
        continue
    if allgoodfactors(number):
        count += 1
        print('Success on', number, '!')
        # if not sp.isprime(number):
        #     print(number, 'is composite')

print('Final number of solutionable integers is:', count)

Small ok primes dictionary loaded...
The number of =1mod3 primes between 10^19 and 10^19 + 10^4 is 130 .
A new exceptional OK number! u = 3161700011 ; v = 11631758 ; n = u^2 + 27v^2 = 10^19 + 5349
Oh dang and it's even a prime! Nice one, 10^19 + 5349 !
Exceptional OK prime count: 1
A new exceptional OK number! u = 3161583656 ; v = 12749417 ; n = u^2 + 27v^2 = 10^19 + 3339
Oh dang and it's even a prime! Nice one, 10^19 + 3339 !
Exceptional OK prime count: 2
A new exceptional OK number! u = 3161272207 ; v = 15345446 ; n = u^2 + 27v^2 = 10^19 + 1581
A new exceptional OK number! u = 3160521604 ; v = 20278789 ; n = u^2 + 27v^2 = 10^19 + 8883
Oh dang and it's even a prime! Nice one, 10^19 + 8883 !
Exceptional OK prime count: 3
A new exceptional OK number! u = 3152956274 ; v = 46693141 ; n = u^2 + 27v^2 = 10^19 + 1863
Oh dang and it's even a prime! Nice one, 10^19 + 1863 !
Exceptional OK prime count: 4
A new exceptional OK number! u = 3144739538 ; v = 64006145 ; n = u^2 + 27v^2 = 10^19 + 1119